# 乳癌資料庫預測：SVM（SVR）分類示範

> 資料集：`sklearn.datasets.load_breast_cancer()`

> 模型：`sklearn.svm.SVR`（支援向量回歸）

⚠️ **注意**：SVR 原本是「回歸」模型，不是分類器。這裡用常見做法：
- 先用 SVR 輸出連續分數（接近 0 或 1）
- 再用閾值（例如 0.5）把分數轉為二元分類

若想用「真正的 SVM 分類器」，通常會用 `SVC` / `LinearSVC`。

## Step1. 下載資料（匯入內建乳癌資料集）

In [ ]:
from sklearn import datasets
import numpy as np
import pandas as pd

# 下載乳癌資料集
cancer = datasets.load_breast_cancer()

X = cancer.data
y = cancer.target  # 0=malignant(惡性), 1=benign(良性)

print("Features shape:", X.shape)
print("Target shape:", y.shape)
print("Feature names (前10個):", cancer.feature_names[:10])
print("Target names:", cancer.target_names)

## Step2. 區分訓練集與測試集

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)

## Step3. 建模（SVR + 特徵標準化）

SVR 對特徵尺度很敏感，因此通常需要 **StandardScaler**。

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

# 建立 pipeline：標準化 + SVR
model = Pipeline([
    ("scaler", StandardScaler()),
    ("svr", SVR(kernel="rbf", C=10.0, epsilon=0.1, gamma="scale"))
])

model

## Step4. 預測

In [ ]:
# 訓練模型
model.fit(X_train, y_train)

# SVR 輸出為連續值（分數）
y_score = model.predict(X_test)

# 將分數轉成分類（閾值 0.5）
y_pred = (y_score >= 0.5).astype(int)

# 看幾筆結果
pd.DataFrame({
    "y_true": y_test[:10],
    "y_score": np.round(y_score[:10], 4),
    "y_pred": y_pred[:10]
})

## Step5. 準確度分析

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
auc = roc_auc_score(y_test, y_score)  # 用連續分數計算 ROC AUC

print(f"Accuracy = {acc:.4f}")
print(f"ROC AUC  = {auc:.4f}")
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=cancer.target_names))

## Step6. `matplotlib.pyplot` 畫圖

這裡畫兩種圖：
1) **ROC Curve**（用 SVR 的分數當作分類分數）
2) **PCA 2D 視覺化**（將 30 維特徵降到 2 維後畫散點圖）

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.decomposition import PCA

# --- (1) ROC Curve ---
fpr, tpr, thresholds = roc_curve(y_test, y_score)

plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle="--")
plt.title("ROC Curve (SVR scores)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.grid(True)
plt.show()

# --- (2) PCA 2D scatter ---
# 先把整體資料做標準化後再 PCA（只用訓練集fit，避免資料外洩）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=2, random_state=42)
X_test_2d = pca.fit_transform(X_test_scaled)

plt.figure(figsize=(7, 5))
# 用真實標籤上色
for label, name in enumerate(cancer.target_names):
    idx = (y_test == label)
    plt.scatter(X_test_2d[idx, 0], X_test_2d[idx, 1], label=f"True: {name}", alpha=0.7)

# 用預測結果畫出錯誤點（紅圈標記）
wrong = (y_pred != y_test)
plt.scatter(X_test_2d[wrong, 0], X_test_2d[wrong, 1], facecolors="none", edgecolors="red", s=120, linewidths=1.5, label="Wrong predictions")

plt.title("Breast Cancer Test Set (PCA 2D) + Wrong Predictions")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.grid(True)
plt.show()

## 補充：如果你想比較 SVC（真正的分類 SVM）

下面這格可選做：把 SVR 換成 `SVC(probability=True)`，比較 Accuracy / AUC。

In [ ]:
from sklearn.svm import SVC

svc_model = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(kernel="rbf", C=10.0, gamma="scale", probability=True, random_state=42))
])

svc_model.fit(X_train, y_train)
svc_prob = svc_model.predict_proba(X_test)[:, 1]
svc_pred = (svc_prob >= 0.5).astype(int)

svc_acc = accuracy_score(y_test, svc_pred)
svc_auc = roc_auc_score(y_test, svc_prob)

print(f"SVC Accuracy = {svc_acc:.4f}")
print(f"SVC ROC AUC  = {svc_auc:.4f}")